## this sample script takes continuous variable as outcome, as example

In [1]:
import os
import pandas as pd

import torch
from torch.utils.data import DataLoader

from gliopath.train.task.long import seed_torch, train, EmbeddingDataset, TaskHead, collate_fn_with_padding, SeriesHead
from gliopath.utils.proces import split_dataset
from gliopath.train.gadget import get_sampler

os.chdir('F:/workspace/pathology/gigapath')
# os.chdir('/mnt/f/workspace/pathology/gigapath')

In [2]:
seed = 42
dataset_df = pd.read_table('data\\metadata.tbl', sep='\t')
test_df = pd.read_table('data\\metadata_test.tbl', sep='\t')
embed_path = 'output/tiles/rand_embed/'
z_score = False
outcome_col = ['IDH1','TP53','ATRX','PTEN','EGFR','TERT']
num_classes = len(outcome_col)
batch_size = 4
num_workers = 0
embed_dim = 1536
weighted_sampler = True
feat_layers=[0,1,2]
num_epochs = 1

splits = ['train', 'val', 'test']
split_col = 'split_col'
id_col = 'id'
params = {
    'lr': 0.001,
    'min_lr': 0.0,
    'num_epochs': num_epochs,
    'eval_interval': 10,
    'output_dir': 'output/models/life',
    'optim': 'sgd',
    'weight_decay': 0.01,
    'outcome_type': 'cat',
    'gc_step': 10,
    'freeze_longnet': True,
}

In [3]:
# set the random seed
seed_torch(torch.device('cuda'), 0)
# read the metadata
dataset_df = split_dataset(dataset_df, id_col='id', type_col='tumour_type', val_split=0.2, test_split=0, in_df=True, split_col='split_col')
test_df['split_col'] = 'test'
dataset_df = pd.concat([dataset_df, test_df], ignore_index=True)

# load the dataset
train_dataset, val_dataset, test_dataset = [EmbeddingDataset(dataset_df, embed_path, feat_layer=feat_layers, split_col=split_col, split=split, id_col=id_col, type_col=outcome_col, outcome_type='gene', z_score=z_score) for split in splits]

# set num_classes
print(f'Sample size:\nTrain: {len(train_dataset)}\tVal: {len(val_dataset)}\tTest: {len(test_dataset)}')

F:\workspace\pathology\gigapath\code\transfer\gliopath\train\task\long.py:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sample_data = torch.load(path)
F:\workspace\patho

Sample size:
Train: 158	Val: 42	Test: 50


In [4]:
# infinite sampler for training
# not sure if cha nge shuffle to TRUE? (*)
# train_sampler = torch.utils.data.sampler.RandomSampler(train_dataset, replacement=True)
train_sampler = get_sampler(train_dataset)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=collate_fn_with_padding, sampler=train_sampler, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=collate_fn_with_padding, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=collate_fn_with_padding, pin_memory=True)

In [5]:
# set the model
import timm
import gliopath.models.longn
from gliopath.models.load import giga_slide_enc
model_longnet = giga_slide_enc(path='model/pub/slide_encoder.pth', global_pool=True)
model = SeriesHead(LongNetModel=model_longnet,
                   TaskHead=TaskHead(768 * len(feat_layers), num_classes),
                   feat_layers=feat_layers)

F:\conda\gliopath\Lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
F:\conda\gliopath\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


F:\workspace\pathology\gigapath
dilated_ratio:  [1, 2, 4, 8, 16]
segment_length:  [np.int64(1024), np.int64(5792), np.int64(32768), np.int64(185363), np.int64(1048576)]
Number of trainable LongNet parameters:  85148160
Global Pooling: True
 Pretrained weights not found at local-dir:model/pub/slide_encoder.pth. Randomly initialized the model! 
Slide encoder param # 86330880


In [56]:
model = SeriesHead(LongNetModel=model_longnet,
                   TaskHead=TaskHead(768 * len(feat_layers), num_classes),
                   feat_layers=feat_layers)

In [7]:
import torch.nn as nn

In [8]:
device = torch.device('cuda')
criterion = nn.BCEWithLogitsLoss()
total_loss = 0

In [9]:
pred_gather, category_gather = [], []
for _, batch in enumerate(val_loader):
    embed, coords, category = batch['tile_embeds'].to(device), batch['coords'].to(device), batch[
        'categories'].to(device)
    with torch.cuda.amp.autocast():
        output = model(embeddings=embed, coords=coords)
        loss = criterion(output, category)
        total_loss += loss.item()
    # gather the predictions and categories
    pred_gather.append(output.cpu().numpy())
    category_gather.append(category.cpu().numpy())
    break

C:\Users\Ge Yi-Lun\AppData\Local\Temp\ipykernel_4332\1005242074.py:5: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [12]:
embed

tensor([[[-8.4078e-01,  1.1254e+00, -2.6722e-01,  ...,  1.6076e+00,
           1.3396e+00,  3.3181e-01],
         [ 7.1748e-01,  4.8242e-01,  1.8288e-01,  ..., -2.4936e-01,
          -7.7294e-01,  1.2160e-01],
         [-1.0970e+00,  1.5350e+00,  2.0763e-01,  ...,  9.6836e-03,
          -1.1984e+00, -5.4515e-01],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 4.4068e-01,  2.0757e+00, -7.5122e-01,  ...,  2.4497e+00,
           5.4724e-01, -2.9873e-02],
         [ 1.2084e+00,  7.6544e-01, -2.5682e-01,  ..., -7.0432e-02,
           1.9836e-01,  6.4339e-01],
         [ 3.8106e-01,  4.5143e-01,  2.2973e-01,  ...,  7.2868e-01,
           1.8203e+00, -1.6289e+00],
         ...,
         [ 0.0000e+00,  0

In [21]:
from tqdm.notebook import tqdm

In [84]:
model = SeriesHead(LongNetModel=model_longnet,
                   TaskHead=TaskHead(768 * len(feat_layers), num_classes, lite=False, long=True),
                   feat_layers=feat_layers)

In [85]:
model.to(device)
model.train()
total_loss = 0.0
num_batches = 0
gc_step = 1
for param in model.longnetmodel.parameters():
    param.requires_grad = False
param_groups = [
    {'params': list(model.taskhead.parameters()), 'lr': 0.0001}
]
optimizer = torch.optim.Adam(param_groups, weight_decay=0.0001)
optimizer.zero_grad()
loss_fn = nn.BCEWithLogitsLoss()

for batch_idx, batch in enumerate(tqdm(train_loader)):
    embed, coords, category = batch['tile_embeds'].to(device), batch['coords'].to(device), batch['categories'].to(device)

    # Regular training
    with torch.cuda.amp.autocast():
        output = model(embeddings=embed[:,:200,:], coords=coords[:,:200,:])
        loss = loss_fn(output, category)
        loss = loss / gc_step
    loss.backward()

    if (batch_idx + 1) % gc_step == 0:
        optimizer.step()
        optimizer.zero_grad()

    total_loss += loss.item() * gc_step
    num_batches += 1

    break
    if batch_idx>1:
        break

avg_loss = total_loss / num_batches


  0%|          | 0/40 [00:00<?, ?it/s]

C:\Users\Ge Yi-Lun\AppData\Local\Temp\ipykernel_4332\410019691.py:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


In [82]:
embed = torch.where(embed == 0, torch.tensor(1e-8), embed)

In [83]:
with torch.cuda.amp.autocast():
    output = model(embeddings=embed, coords=coords)
output

C:\Users\Ge Yi-Lun\AppData\Local\Temp\ipykernel_4332\871105059.py:1: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


tensor([[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]], device='cuda:0', dtype=torch.float16,
       grad_fn=<AddmmBackward0>)

In [80]:
embed

tensor([[[ 0.1203,  0.1157, -0.3988,  ..., -0.1207, -1.1815, -1.6982],
         [ 0.1887,  0.4822,  0.1379,  ...,  0.4477,  0.9428,  0.5185],
         [-1.0728, -1.2576, -0.4464,  ...,  0.2475,  0.3290,  2.3571],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 1.5056, -0.4710,  0.6082,  ..., -0.5095,  0.7896,  0.5461],
         [-0.4915, -0.0946, -0.9899,  ..., -0.2051,  1.1551, -0.2982],
         [ 0.9325, -0.2584,  1.1333,  ...,  0.1947, -1.2200, -1.3275],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 1.3312, -0.1151,  0.9996,  ..., -0.4596, -0.2778, -0.9568],
         [ 0.0515, -1.6681, -1.4083,  ...,  0

In [76]:
embed

tensor([[[ 0.5879,  1.3860, -0.6725,  ...,  0.6962, -1.6394,  0.3380],
         [ 1.4756, -0.7282,  1.3812,  ..., -0.6527, -0.5165,  1.3156],
         [-0.0662, -0.1814,  0.2001,  ..., -0.9326, -0.1593,  0.9621],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.9276, -0.2705, -0.8283,  ...,  0.9314,  0.7463, -1.1774],
         [ 0.9273, -0.7051, -0.6965,  ...,  1.0321, -1.1501,  0.1631],
         [ 1.6544, -0.8149,  1.4631,  ..., -0.3226, -0.9224, -1.5511],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-1.6390, -0.8477,  0.9876,  ..., -0.1103, -1.3053, -0.2323],
         [ 0.6189, -0.8792,  0.1409,  ..., -0

In [86]:
output

tensor([[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]], device='cuda:0', dtype=torch.float16,
       grad_fn=<AddmmBackward0>)

In [52]:
embed.shape

torch.Size([4, 300, 1536])

In [53]:
coords.shape

torch.Size([4, 300, 2])

In [25]:
batch_idx

39

In [37]:
embed[:,:200,:].shape

torch.Size([4, 200, 1536])

In [78]:
with torch.cuda.amp.autocast():
    output = model(embeddings=embed[:,:10,:], coords=coords[:,:10,:])
output

C:\Users\Ge Yi-Lun\AppData\Local\Temp\ipykernel_4332\470183074.py:1: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


tensor([[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]], device='cuda:0', dtype=torch.float16,
       grad_fn=<AddmmBackward0>)

In [35]:
embed.shape

torch.Size([4, 282, 1536])

In [34]:
coords

tensor([[[  3072., 182656.],
         [  3072.,  47232.],
         [  3072., 107648.],
         ...,
         [     0.,      0.],
         [     0.,      0.],
         [     0.,      0.]],

        [[  3072.,  17792.],
         [  3072.,  82560.],
         [  3072.,  46208.],
         ...,
         [     0.,      0.],
         [     0.,      0.],
         [     0.,      0.]],

        [[  3072., 180096.],
         [  3072., 119424.],
         [  3072., 209536.],
         ...,
         [     0.,      0.],
         [     0.,      0.],
         [     0.,      0.]],

        [[  3072., 232320.],
         [  3072.,  67456.],
         [  3072., 235392.],
         ...,
         [  3072.,  61568.],
         [  3072.,  80000.],
         [  3072.,  95360.]]], device='cuda:0')

In [33]:
embed

tensor([[[ 0.1935, -1.4096,  0.3916,  ..., -0.6122, -2.1608,  0.1298],
         [ 0.7913, -0.0469,  0.1334,  ..., -0.8014,  0.6772, -1.1878],
         [-0.3871,  2.1709,  1.3291,  ..., -0.4214, -0.3425,  0.6355],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-3.4571, -1.1231,  1.0056,  ...,  0.9908,  2.0672,  0.0549],
         [-1.2838, -0.4897,  0.8906,  ..., -0.7166, -0.1174, -0.4543],
         [ 0.6704, -0.3541,  0.7100,  ..., -1.7715, -1.1464, -1.3441],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.0729, -0.0437, -1.6062,  ...,  0.0264, -0.2498,  1.2771],
         [ 1.1751, -0.0650, -0.3351,  ..., -0

In [32]:
output

tensor([[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]], device='cuda:0', dtype=torch.float16,
       grad_fn=<AddmmBackward0>)

In [31]:
total_loss

nan

In [55]:
# Train the model
pred_gather, target_gather = train(model, train_loader, val_loader, test_loader, **params)

LongNet encoder has been frozen
Set the optimizer as sgd
Start training for 1 epochs with gradient accumulation steps: 10


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 0:   0%|          | 0/40 [00:00<?, ?it/s]

F:\workspace\pathology\gigapath\code\transfer\gliopath\train\task\long.py:476: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


RuntimeError: mat1 and mat2 shapes cannot be multiplied (4x2304 and 96x6)